# Discovering and Visualizing Topics in Texts

Most typical cases of text classification in NLP (named entity recognition, question answering, etc) require training datasets where each piece of text is associted with a label. However, in real-life scenarios, text collections rarely come with metadata labels that tell you what the texts are about. When people answer open-ended survey questions, for example, they don't tag their answer with the topics they discuss.

**Topic modeling** is an unsupervised classification technique that is able to discover the topics in a collection of texts by looking at their commonalities. In this context, "topics" refers to groups of related words that often occur together in the same text. For example, in a collection of newspaper articles a topic model may identify one topic that is made up of words such as "politician", "law", and "parliament", and another characterized by words such as "player", "match" and "penalty". Topic models only go as far as identifying clusters of related words; a human is still needed to interpret these clusters and give them labels such as "politics" and "football". 

One of the most popular topic models is Latent Dirichlet Allocation (LDA). LDA is a generative model that sees every text as a mixture of topics and each sentence as a mixture of words. For example, the "football" topic will generate the word "penalty" with a high probability, while the "politics" topic will have a much higher probability for "politician" than for "penalty". Other words, such as "the" and "an", will have similar probabilities in all topics. LDA takes its name from the Dirichlet probability distribution. This is the prior distribution it assumes the topics in a text will have.

Modified from https://github.com/nlptown/nlp-notebooks/blob/master/Discovering%20and%20Visualizing%20Topics%20in%20Texts%20with%20LDA.ipynb

## Data

Insight fellows frequently come up with project ideas that revolve around topic modeling of online reviews. For this example, we'll use a curated dataset of Amazon reviews for toys and games filtered to products with more than 500 reviews. This dataset was collected and cleaned in the `0.Data Scraping and Prep` notebook.

In [1]:
import pandas as pd
import numpy as np

# This command will unzip the text file
# The rest of the cell will run even if this command fails
!gzip -d ../data/reviews_amazon_small.csv.gz

reviews = pd.read_csv('../data/reviews_amazon_small.csv')

Each of the rows in this data frame contains a product identifier, a product title, a rating left by a user, the text of their review, and a clean version of the text:

In [2]:
reviews.head()

,product_id,product_title,rating,text,text_processed
0,0975277324,Ticket To Ride,4.0,Why not 5 stars? THis game only got 4 due to t...,why not stars this game only got due to the ...
1,0975277324,Ticket To Ride,5.0,One of the best board games around This is an ...,one of the best board games around this is an ...
2,0975277324,Ticket To Ride,5.0,One of our favorites of all time! What are you...,one of our favorites of all time what are you ...
3,0975277324,Ticket To Ride,5.0,Great Game This is my families first experienc...,great game this is my families first experienc...
4,0975277324,Ticket To Ride,4.0,A Ticket to Fun I didn't think that I was goin...,a ticket to fun i didnt think that i was going...


## Preprocessing

Before we train a topic model, we need to tokenize our texts. Let's do this with the [spaCy](https://spacy.io/) NLP library. We need to load a statisti English and use spaCy to perform our first preprocessing pass:

In [3]:
import spacy

# If you haven't installed the spaCy language model, uncomment the following line and run this cell
# ! python -m spacy download en_core_web_sm

# You will need to restart the notebook (go to the menu Kernel -> Restart) and re-run cells up to this point

In [4]:
nlp = spacy.load('en_core_web_sm')

texts = reviews['text_processed'].tolist()
%time spacy_docs = list(nlp.pipe(texts))

CPU times: user 1min 12s, sys: 15.1 s, total: 1min 27s
Wall time: 1min 28s


The text of each review is now a spaCy Doc that we can transform into a list of tokens. Instead of the original tokens, we're going to work with the **lemmas** instead. This will allow our model to generalize and understand that different forms of a word should be treated as one.

Stemming and Lemmatization both generate the root form of the words. Lemmatization uses the rules about a language and the resulting tokens are all actual words. For example, the word "thought" becomes the lemma "think". Stemming is a crude heuristic that chops off the ends of words such that the resulting tokens may not be actual words. Stemming is faster but only works well for simple words like "toys" and "toy".

This is the full list of our initial preprocessing steps: 
 
- remove all words shorter than 2 characters (these are often fairly uninteresting from a topical point of view)
- drop all stopwords
- lowercase remaining lemmas

In [5]:
docs = [[t.lemma_.lower() for t in doc if len(t.orth_) > 2 and not t.is_stop] for doc in spacy_docs]
print(docs[:3])

[['star', 'game', 'get', 'luck', 'factor', 'matter', 'play', 'draw', 'bad', 'card', 'everytime', 'wonderful', 'wonderful', 'game', 'perfect', 'game', 'people', 'eurogame', 'like', 'settler', 'catan', 'carcassonne', 'easy', 'explain', 'minute', 'top', 'fast', 'play', 'hour', 'good', 'strategy', 'good', 'theme', 'feel', 'like', 'educate', 'little', 'bit', 'area', 'geographybottom', 'line', 'good', 'family', 'board', 'game', 'existance', 'learn', 'play', 'relatively', 'quickly'], ['good', 'board', 'game', 'excellent', 'board', 'game', 'reasonable', 'mix', 'skill', 'luck', 'involve', 'depend', 'people', 'play', 'quickly', 'turn', 'game', 'thirty', 'minute', 'hour', 'rule', 'simple', 'experienced', 'player', 'advantage', 'beginner', 'people', 'speed', 'turn', 'game', 'strategy', 'win', 'consistently', 'gain', 'gamesthere', 'version', 'expansion', 'game', 'fun', 'interesting', 'worth', 'look', 'good', 'suit', 'group', 'player', 'basic', 'form', 'game', 'straightforward', 'set', 'rule', 'tick

Next, we also want to take frequent bigrams into account. **Bigrams are multiword units**, such as "colored pencil" that actually form one word rather than two. We'll use Gensim to first identify the frequent bigrams in the corpus, then append them to the list of tokens for the documents in which they appear. This means the bigrams will not be in their correct position in the text, but that's fine: topic models are bag-of-word models that ignore word position anyway.

In [6]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)
tokens = []

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:  # bigrams can be recognized by the "_" that joins the invidual words
            docs[idx].append(token)
            tokens.append(token)
            
print(list(set(tokens))[:10])

['player_perform', 'living_room', 'take_beating', 'old_brother', 'car_ride', 'judge_choose', 'write_review', 'stuff_animal', 'chute_ladder', 'photo_video']


Next, we move on to the final Gensim-specific preprocessing steps. First, we create a dictionary representation of the documents. This dictionary will map each word to a unique ID and help us create bag-of-word representations of each document. These bag-of-word representations contain the ids of the words in the document, together with their frequency. Additionally, we can remove the least and most frequent words from the vocabulary. This improves the quality of our topic model and speeds up its training. The minimum frequency of a word is expressed as an absolute number, the maximum frequency is the proportion of documents a word is allowed to occur in.

In [7]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in original documents:', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Number of unique words after removing rare and common words:', len(dictionary))

print("Example representation of document 3:", dictionary.doc2bow(docs[2]))

Number of unique words in original documents: 20934
Number of unique words after removing rare and common words: 6083
Example representation of document 3: [(3, 1), (5, 1), (8, 1), (13, 1), (38, 1), (52, 1), (67, 1), (72, 1), (80, 1), (82, 1), (89, 1), (92, 1), (96, 1), (97, 1), (117, 1), (118, 1), (121, 1), (124, 1), (125, 1), (126, 1), (127, 1), (128, 1), (129, 1), (130, 1), (131, 1), (132, 2), (133, 1), (134, 1), (135, 1), (136, 1), (137, 2), (138, 1), (139, 1), (140, 2), (141, 1), (142, 1), (143, 1), (144, 1), (145, 1), (146, 4), (147, 1), (148, 1), (149, 1), (150, 1)]


Then we create bag-of-word representations for each document in the corpus:

In [8]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

## Training

Now it's time to train our topic model. We do this with the following parameters: 

- `corpus`: the bag-of-word representations of our documents
- `id2token`: the mapping from indices to words
- `num_topics`: the number of topics we want the model to identify
- `chunksize`: the number of documents the model sees for every update
- `passes`: the number of times we show the total corpus to the model during training
- `random_state`: we use a seed to ensure reproducibility.

On a corpus of this size, the training will typically about a minute.

In [9]:
from gensim.models import LdaModel

%time model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=500, passes=3, random_state=1)

CPU times: user 9.65 s, sys: 33.6 ms, total: 9.68 s
Wall time: 10.4 s


## Results

Let's take a look at what the model has learnt. We do this by printing out the ten words that are most characteristic for each of the topics. Most topics show common words like "play", "love" and "game" but it's hard to identify any other patterns in the data.

In [10]:
for (topic, words) in model.print_topics():
    print(topic+1, ":", words, '\n')

1 : 0.033*"year_old" + 0.025*"gift" + 0.020*"get" + 0.019*"christmas" + 0.018*"recommend" + 0.017*"purchase" + 0.017*"daughter" + 0.013*"son" + 0.012*"highly" + 0.012*"come" 

2 : 0.038*"leapfrog" + 0.025*"leap" + 0.022*"book" + 0.022*"child" + 0.021*"learn" + 0.018*"ipad" + 0.018*"read" + 0.016*"daughter" + 0.016*"picture" + 0.014*"educational" 

3 : 0.029*"leappad" + 0.026*"battery" + 0.016*"fly" + 0.013*"time" + 0.012*"use" + 0.012*"charge" + 0.011*"work" + 0.010*"helicopter" + 0.009*"get" + 0.009*"pad" 

4 : 0.030*"button" + 0.024*"sound" + 0.023*"daughter" + 0.020*"loud" + 0.016*"cute" + 0.016*"ball" + 0.014*"noise" + 0.013*"push" + 0.013*"little" + 0.011*"think" 

5 : 0.028*"tablet" + 0.020*"device" + 0.016*"screen" + 0.013*"year_old" + 0.012*"car" + 0.012*"video" + 0.010*"learning" + 0.010*"little" + 0.010*"use" + 0.009*"song" 

6 : 0.079*"app" + 0.022*"family" + 0.016*"adult" + 0.015*"lot" + 0.013*"time" + 0.013*"easy" + 0.013*"young" + 0.013*"find" + 0.013*"enjoy" + 0.013*"goo

Another way of inspecting the topics is by visualizing them. This can be done with the [pyLDAvis](https://github.com/bmabey/pyLDAvis) library. PyLDAvis will show us how popular the topics are in our corpus, how similar the topics are, and which are the most salient words for this topic. Note it's important to set `sort_topics=False` on the call to pyLDAvis. If you don't, it will order the topics differently than Gensim. 

In [11]:
import pyLDAvis.gensim
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

/Users/mperignon/anaconda3/envs/NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.036972  0.027781       1        1  12.500275
1      0.046160  0.075046       2        1   9.772259
2      0.015578  0.022003       3        1  19.261728
3      0.101479  0.176329       4        1   7.665074
4      0.052124  0.044874       5        1  10.310793
5      0.059023 -0.232116       6        1   7.427310
6      0.065701 -0.222066       7        1  11.075222
7     -0.105420  0.055121       8        1   7.772488
8      0.100428  0.060878       9        1  10.749910
9     -0.372046 -0.007850      10        1   3.464945, topic_info=     Category         Freq      Term        Total  loglift  logprob
5     Default  2864.000000      card  2864.000000  30.0000  30.0000
1105  Default  1996.000000       app  1996.000000  29.0000  29.0000
5866  Default  1111.000000      loom  1111.000000  28.0000  28.0000
3607  Default  1255.000000  leapfrog  1255.000000  27.0000  27.0000
100   Default  1638.000000       set  1638.000000  26.0000  26.0000
...       ...          ...       ...          ...      ...      ...
564   Topic10    79.858398    amazon   497.496796   1.5331  -4.9968
81    Topic10    89.807472      look  1181.918823   0.7852  -4.8793
994   Topic10    74.632950      real   707.733093   1.1130  -5.0644
447   Topic10    69.008202    design   372.417542   1.6767  -5.1428
42    Topic10    68.328773     basic   222.730835   2.1808  -5.1527

[642 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
374       1  0.073749         able
374       2  0.022445         able
374       3  0.203613         able
374       4  0.040081         able
374       5  0.333476         able
...     ...       ...          ...
557       8  0.030312  young_child
557       9  0.636551  young_child
1150      1  0.807737      youtube
1150      5  0.187159      youtube
2684      2  0.991769       zipper

[1536 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Finally, let's inspect the topics the model recognizes in some of the individual documents. Here we see how LDA tends to assign a high probability to a low number of topics for each documents, which makes its results easily interpretable.

In [12]:
for (text, doc) in zip(texts[:20], docs[:20]):
    print(text)
    print('-'*10)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.3])
    print('\n')

why not  stars this game only got  due to the luck factor  no matter how well you play you could just draw bad cards everytime  that is it  other than that this is a wonderful wonderful game  this is the perfect game to get people into eurogames like settlers of catan and carcassonne  easy to explain   minutes tops  fast to play   hour  good strategy  good theme  you even feel like you are being educated at least a little bit in the area of us geographybottom line  best family board game in existance  anyone can learn to play it relatively quickly
----------
[]


one of the best board games around this is an excellent board game with a reasonable mix of skill and luck involved  depending on how many people are playing and how quickly they take their turns this game can take anywhere between thirty minutes and two hours  the rules are simple and while experienced players have an advantage against beginners people can get up to speed after a few turns of their first game  strategies for 

Looping through all texts, let's save the most likely topic number.

In [13]:
topic_nums = []
for (text, doc) in zip(texts, docs):
    probs = np.array(model[dictionary.doc2bow(doc)])
    topic_nums.append(probs[np.argsort(probs[:,-1])][-1,0])
    
reviews['topic'] = topic_nums

product_vs_topic = pd.crosstab(reviews['product_title'], reviews['topic'])
product_vs_topic = product_vs_topic.T / product_vs_topic.sum(axis = 1) * 100
product_vs_topic = product_vs_topic.T

In [14]:
import matplotlib.pyplot as plt
from matplotlib import colors
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

product_vs_topic.round(2).style.apply(background_gradient,
               cmap='YlGnBu',
               m=product_vs_topic.min().min(),
               M=product_vs_topic.max().max(),
               low=0.5,
               high=0.8)

topic,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
product_title,,,,,,,,,,
Apples to Apples Party Box - The Game of Crazy Combinations (Family Edition),5.83,1.67,0,0,2.5,25,61.67,0,3.33,0
Battat Take-A-Part Airplane,23.44,1.56,7.29,0,53.65,1.04,1.04,4.69,6.77,0.52
Campfire Kids Campfire,12,2,27,32,20,1,1,1,4,0
Cards Against Humanity,11.33,0.65,4.21,0,0.97,14.56,66.34,0.32,0.97,0.65
Cards Against Humanity: First Expansion,7.09,1.57,0.79,0.79,0.79,4.72,81.89,1.57,0.79,0
Cards Against Humanity: Second Expansion,6.42,0.92,2.75,0.92,0,6.42,79.82,0.92,1.83,0
"Educational Insights The Sneaky, Snacky Squirrel Game",23.58,20.33,0.81,1.63,7.32,10.57,8.94,4.88,20.33,1.63
Fisher Price Learning Home,20.95,0.95,4.76,27.62,2.86,0.95,0,0,41.9,0
"Fisher-Price Brilliant Basics Baby""s First Blocks",7.73,5.15,4.12,8.25,3.61,2.58,1.03,19.59,46.39,1.55


## Conclusions

Many collections of unstructured texts don't come with any labels. Topic models such as Latent Dirichlet Allocation are a useful technique to discover the most prominent topics in such documents. Gensim makes training these topics model easy, and pyLDAvis presents the results in a visually attractive way. Together they form a powerful toolkit to better understand what's inside large sets of documents and to explore subsets of related texts. However, these methods can perform poorly in short texts with vague or unspecified subjects. Although traditional topic models are lacking in more semantic information (they don't use word embeddings, for instance), they can be really quick way of getting insights into large collections of documents.